# Exploring Eco topics with Python | TEST NOTEBOOK - Carbon Polluters Exploration
---

**<font color='red'>FYI internet access required for E.</font>**

---

## A.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
raw_data = pd.read_csv("CarbonPollutersExp_DATA.csv")

In [ ]:
df_prep = raw_data.copy()

## B.

In [ ]:
df_prep.shape

## C.

In [ ]:
list(enumerate(df_prep.columns))

In [ ]:
df_prep.drop(columns = df_prep.columns[[10,22,23,24]], inplace=True)

In [ ]:
df_prep.dtypes

In [ ]:
df_prep.rename(columns = lambda x: x.replace("TOTAL REPORTED EMISSIONS, ", ""), inplace=True)

In [ ]:
df_prep.loc[:, "2011":] = df_prep.loc[:, "2011":].applymap(lambda x: x.replace(",", ""))

In [ ]:
df_prep.replace("---", np.nan, inplace=True)

In [ ]:
df_prep.loc[:, "2011":] = df_prep.loc[:, "2011":].apply(pd.to_numeric)

In [ ]:
df_prep.describe()

In [ ]:
df_prep[ df_prep.columns[-11:]].T.plot(legend=False)

In [ ]:
((df_prep.iloc[:, -11:].isna()) | (df_prep.iloc[:, -11:] > 25000)).values.all()

In [ ]:
df_prep["Cumulative"] = df_prep.loc[:, "2011":].sum(axis=1)

In [ ]:
df_prep["Cumulative"].sum()

## D.

In [ ]:
import geopandas as gpd

In [ ]:
geo_df = gpd.GeoDataFrame(df_prep, geometry=gpd.points_from_xy(df_prep["LONGITUDE"], df_prep["LATITUDE"]))

In [ ]:
geo_df.plot(column="Cumulative", cmap="cool")

In [ ]:
base_map = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres")) 

In [ ]:
base_map.plot()

In [ ]:
base_map.at[4, "geometry"]

In [ ]:
base_map.crs

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))    
base_map.plot(ax=ax)
geo_df.plot(ax=ax, column="Cumulative", cmap="autumn_r")  
plt.show()

## E.

In [ ]:
import folium

In [ ]:
US_state_boundaries = gpd.read_file("cb_2021_us_state_20m/cb_2021_us_state_20m.shp")

In [ ]:
US_state_boundaries.explore()

In [ ]:
base_layer = US_state_boundaries.explore(location=[39, -97], width="50%", height="50%", zoom_start=4, min_zoom=3, tooltip=False, style_kwds=dict(fillOpacity=1, weight=1, fillColor="gainsboro"), highlight_kwds=dict(fillOpacity=0, weight=3))

geo_df.explore(m=base_layer, column="Cumulative", marker_type="circle", marker_kwds=dict(radius=4828, fill=False), cmap="autumn_r", popup=["FACILITY", "Cumulative"], tooltip=False)

base_layer

In [ ]:
from folium.plugins import HeatMap

heatmap_points = [ [lat, long, weight] for lat, long, weight in zip(geo_df["LATITUDE"], geo_df["LONGITUDE"], geo_df["Cumulative"]) ]

map_layer = folium.Map(location=[39, -97], tiles="cartodbpositron", width="50%", height="50%", zoom_start=4, min_zoom=3)
HeatMap(heatmap_points, radius=15, blur=5).add_to(map_layer)

map_layer

## F.

In [ ]:
df_trends = df_prep.copy()

In [ ]:
del df_trends["geometry"]

In [ ]:
df_trends.set_index(["FACILITY", "STATE"], inplace=True)

In [ ]:
df_trends.drop(df_trends.loc[:, "GHGRP ID":"PARENT COMPANIES"], axis=1, inplace=True)

In [ ]:
df_trends.sort_values("Cumulative").loc[:, "2011":"2021"].plot(kind="barh", stacked=True, figsize=(20,15), legend=False, fontsize=8)

In [ ]:
worst12states = df_trends.groupby(["STATE"])["Cumulative"].sum().sort_values(ascending=False).index[:12]

In [ ]:
df_timeseries = df_trends.drop(columns = "Cumulative").copy()

In [ ]:
df_timeseries.xs("PA", level="STATE").T

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(25,20), sharey=True)
for state, ax in zip(worst12states, axes.flatten()):
    df_timeseries.xs(state, level="STATE").T.plot(ax=ax)
    ax.set_title(state)
    ax.legend(loc="best", fontsize="medium", labelspacing=0.25)

plt.setp(axes, xlabel="Reporting Year", ylabel="CO2e metric tons per Reporting Year", xticks=range(len(df_timeseries.columns)), xticklabels= df_timeseries.columns,    
         yticks=range(0,550001,25000))
plt.tight_layout()
plt.show()

---
Copyright © 2023 Rho Zeta AI Ltd. All rights reserved.